## Language Project - Test 3

This note book contains the tests that I ran to generate recipies using grocery reciepts (third example).

In [ ]:
# Mount google drive

from google.colab import drive
drive.mount('/content/gdrive')
nbdir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
# Log in to hugging face

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Install modules

!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Install

## Extracting text from reicpts

The first step is to extract the text from the images of grocery recipts. I am doing this using Easy OCR, a python module for extracting text from image. This tends to recognize the original text better but adds a extra characters.

In [ ]:
# Install modules

! pip install easyocr

! pip install torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 18.0 MB/s eta 0:00:00


In [ ]:
# Set the language to english

import easyocr

reader = easyocr.Reader(['en'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

The dataset that I am using contains images of grocery recipts both with and without bounding boxes around the text. The version with teh bounding boxes performs better at extracting thext from the images.

In [ ]:
# Results using images without bounding boxes
result = reader.readtext("images/9.jpg")

In [ ]:
res = []

for (bbox, text, prob) in result:
    res.append(text)


food_res = " ".join(res)

print(food_res)

Winco Eoods Ji nhe heu 4 Ru 9o124 75137 er : Yol N/YORK TX GGSC 18 FS Fs 7056087677 56 FS GRND B 20167400008 DIET LN 20337400o02 DRBKSHicks 26743coo008 9 R EYE RND 5Tk FP 20146200008 EYE RND 20254300000 18.28 ES BEEF FOR 20323300000 48 PORK 1/2 LOIN 4497921020 B/BEST $ 1590013401 BAR HAMEAT 'FRNK 4590001903 2.88 0/m WINCO NOODLES 7055280303 TNDRBRO"CHiR4BR 3727101318 5.98 Fs KRFT DELUXE MAC 2100065886 3.64 2 0 82 EGGO WAFFLE 3800037435 L/D NUTTY BARS 2430004120 Lid OATMEAL CRM 2430004101 LiNKS MILD 74295503660 FGfREVOFRHQUSE 9222230256 H RESERS POT  SLD 7111719300 DM SPAG SAUCE 2400052363 hunTS MANWICH 2700044212 82 DM PNAPL  CHNKY 2400000164 96 FS 98 SUNBEAN' BUNS 7763306333 1.98 WINco SNDwch_WT  7055200115 91 WINCO HoMo MILK 7055240600 2.48 R EGGS 7003837280 1.37 SUBTOTAL 121.92 TOTAL  TaX 00 IOTAL 121.92 47770ut4 0oo" nincotuade. Cockrel| Dunetny | Etoru anca Cashl , 09/08/14 11;16.24 7045400658 26840100000 KINGS CHIGKEN 7056,096902 BLEND PICSWEET GREENS PICSHEET O06OO 268401C WINGS

In [ ]:
# Results using images with bounding boxes

result = reader.readtext("boxes/9.png")

In [ ]:
res = []

for (bbox, text, prob) in result:
    res.append(text)


food_res = " ".join(res)

print(food_res)



Winca Eoobs Ji Pheheur 806" 4 Ru 90124 75137 er : Yol 14:16,24 7045 WinGs 28 705689 5739 PicSWEL Cenin 7056087672} 26840100uOn CHICKEN 2016740uUUO 5,48 Es DIFT 203374UUUUU 10.87 PORK 26841800000 .91 201 48200000 5,32 EYE RN Sin FP 2014820000U 9.J9_ES EXE RND Sij, 2025430n000 8 24 ES BFEE' FuR Sie# 20323200000 1874 FoRi/ 372 4437921020 6/BFS1 5 BAR MEAT 'FRNK  1590013401 0/MHAM 4470001903 'INCO NOODLES 7055280303 -68 FS TNDRBRD_CHIXBR 3737101318 5.98 Es KRF DEL UXE MAC 2100065886 3.64 FS 2 0 FGGO_WAFFLE 380003/435 1.96 FS Lrd NUTTy BARS 2430004120 1.25 FS LID  OATMEAL CRM 2430004101 1,25 FS LinkS MTLD 74295503660 5.78 FS KEEB_T OWNHOUSE 3010010055 1.98 FS TURKEY_FRANK 4222281210 88 FS RESERS POT SLD 7111719300 98 FS DMSPAG SAUCE 2400052363 71 FS hunTS MANWICH 2700044212 1.64 FS 82 DM PNAPL  CHNKY 2400000164 96 FS 2@ 98 BUNBE AN BLNS_ 7763306333 98 FS NCO SNDWCH 7055200115 91 FS WINCO HOMO MILK   7055240600 2.48 FS EGGS /003837280 .37 FS SUBTOTAL 121.92 TOTAL  TaX 00 IOTAL 121.92 Taetttt

## Identify food items from the text

The next step is to identify the food items from the text extracted from the recipt. To do this, I used a Named Entity Recognition (NER) model that was fine tuned to recognize one entity - FOOD.  

The first model that I tried for this was FoodBaseBERT, a fine-tuned BERT model that is ready to use for Named Entity Recognition of Food entities. Unfortunately this did not produce good results.

In [ ]:
# FOODBaseBERT

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Dizex/FoodBaseBERT")
model = AutoModelForTokenClassification.from_pretrained("Dizex/FoodBaseBERT")

pipe = pipeline("ner", model=model, tokenizer=tokenizer, device="cuda")

ner_entity_results = pipe(food_res)
print(ner_entity_results)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/20.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1602: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/814 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

[{'entity': 'B-FOOD', 'score': 0.37057465, 'index': 4, 'word': '##oo', 'start': 7, 'end': 9}, {'entity': 'B-FOOD', 'score': 0.6025312, 'index': 24, 'word': 'Yo', 'start': 50, 'end': 52}, {'entity': 'B-FOOD', 'score': 0.51171184, 'index': 49, 'word': 'Ce', 'start': 97, 'end': 99}]


In [ ]:
for i in ner_entity_results:
  print(i["word"])

##oo
Yo
Ce


The other model that I tried was roberta-base-food-ner, which is a fine-tuned version of RoBERTa base on a dataset of about 300 hand-labeled recipes. I was able to get better results using this model, after converting the input text to lower case.

In [ ]:
# roberta-base-food-ner

model = AutoModelForTokenClassification.from_pretrained('carolanderson/roberta-base-food-ner')
tokenizer = AutoTokenizer.from_pretrained("roberta-base", add_prefix_space=True)
nlp = pipeline("ner", model=model, tokenizer=tokenizer, device="cuda")


config.json:   0%|          | 0.00/896 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
results = nlp(food_res.lower(), aggregation_strategy="first")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:392: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


In [ ]:
items = []
for i in results:
  print(i["word"])
  items.append(i["word"])

 chicken
 meat
 noodles
 nutty bars
 oatmeal
 turkey_frank
 sauce
 manwich
 chnky
 homo milk
 eggs
 cockra)!


## Generating recipies

The last step is to generate recipies using the the names of food items identified from the grocery recipt.

The first method that I used was a fintuned lora adapter on a dataset of recipies for the llama 3.1 8B Base model. This method works pretty well, however when there are a large number of items in the input list, the output of the model does not follow the format that it was trained to produce.

In [ ]:
# Load the adapter

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "yr82/recipie-lora-llama-2"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

In [ ]:
def format_list(items):
    formatted_list = ', '.join([f'"{item}"' for item in items])
    return f'[{formatted_list}]'

In [ ]:
# Remove duplicates and use only the first 5 items from the list

unique_food = list(dict.fromkeys(items))

unique_food = unique_food[:5]

food_list = format_list(unique_food)

In [ ]:
# Generate recipie

batch = tokenizer(food_list + " ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=512, min_new_tokens=64, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)

output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)


<ipython-input-19-38d3787dac53>:5: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
sections = output_text.split("\n")

for section in sections:
  print(section)


[" chicken", " meat", " noodles", " nutty bars", " oatmeal"] ->:  title: Chicken Noodle Casserole
ingredients: ["1 can cream of chicken soup", "1 can cream of celery soup", "1 can cream of mushroom soup", "1 can chicken", "1 can meat", "1 pkg. noodles", "1 pkg. nutty bars", "1 pkg. oatmeal"]
directions: ["Mix all ingredients together.", "Put in casserole dish.", "Bake at 350\u00b0 for 1 hour."]


The next method that I tried was using a language model that was fine-tuned specifically for recipie generting recipies. Chef Transformer is a fine-tuned version of the T5 language model trained on cooking recipies. This method works well too, however the directions generated do not always make sense.

In [ ]:
from transformers import FlaxAutoModelForSeq2SeqLM
from transformers import AutoTokenizer

MODEL_NAME_OR_PATH = "flax-community/t5-recipe-generation"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH, use_fast=True)
model = FlaxAutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_OR_PATH)

prefix = "items: "

generation_kwargs = {
    "max_new_tokens": 512,
    "min_new_tokens": 64,
    "num_return_sequences": 1,
    "eos_token_id": tokenizer.eos_token_id,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95
}


special_tokens = tokenizer.all_special_tokens
tokens_map = {
    "<sep>": "--",
    "<section>": "\n"
}
def skip_special_tokens(text, special_tokens):
    for token in special_tokens:
        text = text.replace(token, "")

    return text

def target_postprocessing(texts, special_tokens):
    if not isinstance(texts, list):
        texts = [texts]

    new_texts = []
    for text in texts:
        text = skip_special_tokens(text, special_tokens)

        for k, v in tokens_map.items():
            text = text.replace(k, v)

        new_texts.append(text)

    return new_texts

def generation_function(texts):
    _inputs = texts if isinstance(texts, list) else [texts]
    inputs = [prefix + inp for inp in _inputs]
    inputs = tokenizer(
        inputs,
        padding="max_length",
        truncation=True,
        return_tensors="jax"
    )

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        **generation_kwargs
    )
    generated = output_ids.sequences
    generated_recipe = target_postprocessing(
        tokenizer.batch_decode(generated, skip_special_tokens=False),
        special_tokens
    )
    return generated_recipe


tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

flax_model.msgpack:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
# Generate recipie

generated = generation_function(items)[0]

In [ ]:
sections = generated.split("\n")


for section in sections:
  print(section)



 title: homemade chicken broth recipe
 ingredients: 8 cups chicken stock-- mrs. v s bbq sauce, to taste
 directions: place chicken in a stock pot or crock pot.-- pour over the desired amount of water you are going to add to the pot to cover.-- cover and simmer over low heat till chicken is very tender and falling off the bone.-- remove chicken from the stock pot and shred.-- strain the stock into a bowl and skim off any fat.-- bring back to a boil and adjust the amount of chicken stock you ve made.-- reduce heat to low and let simmer until the liquid is reduced to bf, about 15 minutes.-- serve with mashed potatoes and blue cheese salad.


In [ ]:
# Generate using only the first 5 items

generated = generation_function(unique_food)[0]

In [ ]:
sections = generated.split("\n")

for section in sections:
  print(section)

 title: jo s favorite chicken
 ingredients: 8 chicken pieces-- 1 bottle apricot jam-- 1 can soup chicken
 directions: cook chicken.-- take out and pour 1/4 bottle of apatio sauce over chicken. bake for 1 hour at 350.-- serve with cornbread.


The last method that I tried was using OpenAI's API to interface with GPT-4 to generate recipies. This method generates the best results out all the methods that I tried.

In [ ]:
! pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [ ]:
def format_string(items):
    formatted_list = ', '.join([f'"{item}"' for item in items])
    return f'{formatted_list}'

In [ ]:
item_string = format_string(items)

In [ ]:
print(item_string)

" chicken", " meat", " noodles", " nutty bars", " oatmeal", " turkey_frank", " sauce", " manwich", " chnky", " homo milk", " eggs", " cockra)!"


In [ ]:
import os

os.environ["OPENAI_API_KEY"]  = "OPENAI_API_KEY"

In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Give me a short recipe using the following ingredients - " + item_string}
  ]
)

generated = completion.choices[0].message.content

In [ ]:
sections = generated.split("\n")

for section in sections:
  print(section)

Certainly! Here’s a simple recipe using some of your provided ingredients:

**Chicken and Turkey Frank Noodle Stir-Fry**

**Ingredients:**
- Chicken breast or thighs, sliced (use "chicken")
- Turkey franks, sliced (use "turkey_frank")
- Noodles (use "noodles")
- Sauce of your choice (use "sauce")
- Eggs (2, beaten)
- Chunky vegetables (use "chnky"; can be chunky mixed vegetables)
- Milk (use "homo milk", a splash for the eggs)
- Salt and pepper to taste

**Instructions:**

1. **Prepare the noodles**:
   Cook noodles according to package instructions, then drain and set aside.

2. **Cook the chicken and turkey franks**:
   - Heat a bit of oil in a large pan or wok over medium-high heat.
   - Add the sliced chicken and turkey franks.
   - Cook until the chicken is browned and cooked through.

3. **Add the vegetables**:
   - Add your chunky veggies to the pan.
   - Stir-fry until they are tender-crisp.

4. **Mix in the eggs**:
   - Push the cooked ingredients to one side of the pan.
   - 